In [1]:
import tensorflow as tf
import numpy as np
import os
import random

In [2]:
batch_size = 200

In [3]:
filenames_cats = ['/home/shubham/all/train/'+filename for filename in os.listdir("/home/shubham/all/train") if filename.startswith("cat")]
filenames_dogs = ['/home/shubham/all/train/'+filename for filename in os.listdir("/home/shubham/all/train") if filename.startswith("dog")]

In [4]:
filenames = filenames_cats+filenames_dogs
filenames[1]

'/home/shubham/all/train/cat.4606.jpg'

In [5]:
labels = [0 for _ in range(len(filenames_cats))]+[1 for _ in range(len(filenames_dogs))]
len(labels)

25000

In [6]:
#filenames = tf.constant(filenames)
#labels = tf.constant(labels)

In [7]:
#print tf.shape(filenames)

In [8]:
dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))

In [9]:
##Shuffling the datset 
dataset = dataset.shuffle(len(filenames))

In [10]:
def parse_function(filename, label):
    image_string = tf.read_file(filename)
    
    image = tf.image.decode_jpeg(image_string, channels=3)

    #one_hot = tf.one_hot(label, 2)
    image = tf.image.convert_image_dtype(image, tf.float32)

    image = tf.image.resize_images(image, [64, 64])
    return image, label

In [11]:
dataset = dataset.map(parse_function , num_parallel_calls = 4)

In [12]:
#Data augmentation 
def train_preprocess(image, label):
    image = tf.image.random_flip_left_right(image)

    image = tf.image.random_brightness(image, max_delta=32.0 / 255.0)
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)

    # Make sure the image is still in [0, 1]
    image = tf.clip_by_value(image, 0.0, 1.0)

    return image, label

In [13]:
dataset = dataset.map(train_preprocess, num_parallel_calls=50)
#dataset = dataset.concatenate(dataset1)

In [14]:
dataset = dataset.batch(batch_size) 

In [15]:
iter = dataset.make_one_shot_iterator()
x,y = iter.get_next()
# with tf.Session() as sess:
#     for i in range(25000):
#         sess.run(x)

In [16]:
def init_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev= 0.1))

def init_bias(shape):
    return tf.Variable(tf.constant(0.1,shape=shape))

def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides = [1,1,1,1],padding="SAME")

def max_pool(x):
    return tf.nn.max_pool(x,ksize = [1,2,2,1],strides = [1,2,2,1], padding ="SAME")

def convolution_layer(input_x,shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W) + b)

def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer,W) + b

In [ ]:
##Defining the VGG Block
x = tf.reshape(x,shape = [-1,64,64,3])
convo_1 = convolution_layer(x,shape=[3,3,3,64])
convo_2 = convolution_layer(convo_1,shape=[3,3,64,64])
pool_1 = max_pool(convo_2)
fc = tf.reshape(pool_1,shape = [-1,32*32*64])
fc_1 = normal_full_layer(fc,1000)
fc_1 = tf.nn.relu(fc_1)
fc_2 = normal_full_layer(fc_1,2)

logits = tf.nn.softmax(fc_2)

labels = y
labels = tf.cast(labels, tf.int64)
tf.shape(y)
# Define the prediction as the argmax of the scores
predictions = tf.argmax(logits, 1)

# Define the loss
loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

# Create an optimizer that will take care of the Gradient Descent
optimizer = tf.train.AdamOptimizer(0.1)

matches = tf.equal(predictions,labels)
acc = tf.reduce_mean(tf.cast(matches,tf.float32))

# Create the training operation
train_op = optimizer.minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        _, loss_val = sess.run([train_op, loss])
        print loss_val
        print sess.run(acc)

0.84837097
0.49
0.79326165
0.465
0.7882617
0.5
0.77826154
0.565
0.80326164
0.505
0.83326155
0.54
0.82826155
0.54
0.84326166
0.505
0.8232616
0.505
0.75326174
0.445
0.78326166
0.555
0.8182616
0.51
0.90326154
0.475
0.7982616
0.51
0.7782617
0.465
0.77826154
0.515
0.82326156
0.455
0.83326155
0.475
0.8082616
0.53
0.83326155
0.53
0.7782617
0.47
0.80826163


In [ ]:
'''The below code is using the inbuilt layer functions of tensorflow'''

# out = x
# out = tf.layers.conv2d(out, 64, 3, padding='same')
# out = tf.nn.relu(out)
# out = tf.layers.conv2d(out, 64, 3, padding='same')
# out = tf.layers.max_pooling2d(out, 2, 2)

# # First, reshape the output into [batch_size, flat_size]
# out = tf.reshape(out, [-1, 32 * 32 * 64])
# # Now, logits is [batch_size, 6]
# logits = tf.layers.dense(out,1000)
# logits = tf.nn.relu(logits)
# logits = tf.layers.dense(logits,2)
# logits = tf.nn.softmax(logits)

# labels = y
# labels = tf.cast(labels, tf.int64)

# # Define the prediction as the argmax of the scores
# predictions = tf.argmax(logits, 1)

# # Define the loss
# loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

# # Create an optimizer that will take care of the Gradient Descent
# optimizer = tf.train.AdamOptimizer(0.001)

# matches = tf.equal(predictions,labels)
# acc = tf.reduce_mean(tf.cast(matches,tf.float32))

# # Create the training operation
# train_op = optimizer.minimize(loss)

# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     for i in range(100):
#         _, loss_val = sess.run([train_op, loss])
#         print loss_val
#         print sess.run(acc)     